<h1><center>Understanding Tokens</center></h1>
<hr><hr><hr>

Tokenization is a fundamental principle in natural language processing (NLP) that plays a crucial role in enabling language models to comprehend written information. It entails breaking down textual inputs into individual units called tokens, forming the foundation for effectively understanding and processing language by neural networks. In the previous lesson, we introduced the concept of tokens as a means to define the input for language models (LLMs). 

The context length of the model is a frequently discussed characteristic among language models. As an example, the <b><u>GPT-3.5 model has a context length of 4096 tokens, covering both the tokens in the prompt and the subsequent completion</u></b>. Due to this constraint, it is advisable to be mindful of token usage when making requests to language models. Nonetheless, various approaches exist to tackle this challenge when dealing with long-form inputs or multiple documents. These methods include **breaking up a lengthy prompt into smaller segments and sending requests sequentially, or alternatively, submitting independent requests for each document and merging the responses in a final step.**

Now, let's delve deeper into what exactly tokens represent and their significance in this context. The tokenization process involves creating a systematic pipeline for transforming words into tokens. However, it is crucial to provide a clear understanding of what exactly tokens represent in this context. Researchers have incorporated three distinct encoding approaches into their studies. The following figure showcases a sample of each approach for your reference.

1. **Character Level**: Consider each character in a text as a token.
2. **Word Level**: Encoding each word in the corpus as one token.
3. **Subword Level**: Breaking down a word into smaller chunks when possible. For example, we can encode the word “basketball” to the combination of two tokens as “basket” + “ball”.

<img src="./images/02-tokenization encoding approaches.jpg" width="1000px"/>


**Subword-level encoding offers increased flexibility and reduces the number of required unique tokens to represent a corpus**. This approach enables the combination of different tokens to represent new words, eliminating the need to add every new word to the dictionary. This technique proved to be the most effective encoding when training neural networks and LLMs. **<u>Well-known models like GPT family, LLaMA employ this Subword-level encoding tokenization method</u>**. Therefore, our sole focus will be on one of its specific variants, known as **Byte Pair Encoding (BPE)**. It is worth mentioning that other subword level algorithms exist, such as **WordPiece** and **SentencePiece**, which are used in practice. However, we will not delve into their specifics in this discussion. Nevertheless, it is important to note that while their token selection methods may differ, the fundamental process remains the same.

## Byte Pair Encoding (BPE):
------------------------------
It is an iterative process to extract the most repetitive words or subwords in a corpus. The algorithm starts by counting the occurrence of each character and builds on top of it by merging the characters. It is a greedy process that carefully considers all possible combinations to identify the optimal set of words/subwords that covers the dataset with the least number of required tokens.

The next step involves creating the vocabulary for our model, which consists of a comprehensive dictionary comprising the most frequently occurring tokens extracted by BPE (or another technique of your choosing) from the dataset. The definition of a dictionary (`dict` type) is a data structure that holds a key and value pair for each row. In our particular scenario, each data point is assigned a key represented by an index that begins from 0, while the corresponding value is a token.

Due to the fact that neural networks only accept numerical inputs, we can utilize the vocabulary to establish a mapping between tokens and their corresponding IDs, like a lookup table. We have to save the vocabulary for future use cases to be able to decode the model's output from the IDs to words. This is known as a pre-trained vocabulary, an essential component accompanying published pre-trained models. Without the vocabulary, understanding the model's output (the IDs) would be impossible. For smaller models like BERT, the dictionary can consist of as few as 30K tokens, while larger models like GPT-3 can expand to encompass up to 50K tokens.

## Tokens and Cost:
--------------------
There is a direct correlation between the tokens and the cost in most proprietary APIs like OpenAI. It is crucial to highlight that the prices will fall into two categories: the number of tokens in the prompt and the completion (the model's generation), with the completion typically incurring higher costs. For example, at the time of writing this lesson, *GPT-4 will cost 0.03 USD per 1K tokens for processing your inputs, and 0.06 USD per 1K tokens for generation process*. 

We can use:
- The `get_openai_callback` method from LangChain to get the exact cost, or
- Do the tokenization process locally and keep track of number of tokens.

For a rough estimation, as a general rule, OpenAI regards 1K tokens to be approximately equal to 750 words.

## Tokenizers In Action:
---------------------------
In this section, we provide the codes to load the pre-trained tokenizer for the GPT-2 model from the Huggingface Hub using the transformers package. <br>
Before running the following codes, you must install the library using the `pip install transformers` command.

In [2]:
pip show transformers

Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: e:\programs & codes\generative ai\_genai_venv\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
# Download and load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

E:\Programs & Codes\Generative AI\_genai_venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Debanjan Sarkar\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### The above code output may loke like as the one in the image below:

<img src="./images/03-download and load tokenizer output.jpg" width="1200px" />


The code snippet above will grab the tokenizer and load the dictionary, so you can simply use the `tokenizer` variable to encode/decode your text. But before we do that, let's take a look at what the vocabulary contains.

In [5]:
print( tokenizer )

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [6]:
print( tokenizer.vocab )

{'usable': 31979, 'Ġcredibility': 16532, 'Ġomn': 22284, 'Ge': 10082, 'ĠMorrison': 24454, 'Ġdisple': 34421, 'ĠGained': 34775, 'ĠGau': 35094, 'Measure': 47384, 'Ġcrystal': 15121, 'ĠVenus': 21094, 'eting': 13629, 'ĠNigeria': 19398, 'Ġrescued': 19868, 'â': 158, 'Ġ88': 9193, 'Ġfodder': 41579, 'Â¯': 5196, 'Ġreinvent': 36608, 'irms': 8789, 'ĠSAR': 47341, 'Ġcos': 8615, 'Ġpayment': 6074, 'âĢĭ': 9525, 'Ġhovering': 33627, 'arial': 36098, 'Ġhepatitis': 37439, 'Ġdozens': 9264, 'ttle': 23296, 'Ġrepertoire': 42515, 'Ġ291': 43336, 'Ġrather': 2138, 'Ġsolid': 4735, 'Ġunavailable': 23485, 'RIPT': 46023, 'Ġclust': 32966, 'associated': 32852, 'Ġkidneys': 41395, 'Ġbrittle': 49307, '241': 28872, 'ĠMaxim': 38962, 'rell': 11252, 'Global': 22289, 'ĠSix': 9699, 'Ġboats': 16659, 'ĠMX': 26135, 'utm': 26841, 'Ġprovocative': 28695, 'Ġmotiv': 6556, 'Ġhyster': 24258, 'Bonus': 29435, 'ĠGillespie': 45078, 'ĠEverett': 36815, 'Ġdispensaries': 40698, 'gged': 11178, 'Ġargs': 26498, 'Ġentrenched': 32472, 'Ġsings': 33041, 'Ġc

In [13]:
print( "'usable' id: ",tokenizer.vocab["usable"] )
# The output will be: 31979

print( "'optional' id: ",tokenizer.vocab["Ġoptional"] )
# The output will be: 11902

'usable' id:  31979
'optional' id:  11902


- As it can be seen, each entry is a pair of token and ID.
- For example, we can represent the word "usable" with the number 31979.
- You might have noticed a special character, **Ġ**, preceding certain tokens. **This character represents a space**. The next code sample will use the `tokenizer` object to convert a sentence into tokens and IDs.

In [14]:
text_to_encode = "This is a sample text to test the tokenizer."

token_ids = tokenizer.encode( text_to_encode )

In [15]:
print( token_ids )

[1212, 318, 257, 6291, 2420, 284, 1332, 262, 11241, 7509, 13]


In [16]:
print( "Tokens:   ", tokenizer.convert_ids_to_tokens( token_ids ) )
print( "Token IDs:", token_ids )

Tokens:    ['This', 'Ġis', 'Ġa', 'Ġsample', 'Ġtext', 'Ġto', 'Ġtest', 'Ġthe', 'Ġtoken', 'izer', '.']
Token IDs: [1212, 318, 257, 6291, 2420, 284, 1332, 262, 11241, 7509, 13]


- The `.encode()` method can convert any given text into a numerical representation, a list of integers.
- To further investigate the process, we can use the `.convert_ids_to_tokens()` function that shows the extracted tokens.
- As an example, you can observe that the word "tokenizer" has been split into a combination of "token" + "izer" tokens.

## Tokenizers Shortcomings:
------------------------------
Several issues with the present tokenization methods are worth mentioning:

- **Uppercase/Lowercase Words**: The tokenizer will treat the the same word differently based on cases. For example, a word like “hello” will result in token id `31373`, while the word “HELLO” will be represented by three tokens as `[13909, 3069, 46]` which translates to `[“HE”, “LL”, “O”]`.
  
- **Dealing with Numbers**: You might have heard that transformers are not naturally proficient in handling mathematical tasks. One reason for this is the tokenizer's inconsistency in representing each number, leading to unpredictable variations. For instance, the number `200` might be represented as one token, while the number `201` will be represented as two tokens like `[20, 1]`.

- **Trailing whitespace**: The tokenizer will identify some tokens with trailing whitespace. For example a word like `“last”` could be represented as `“ last”` as one token instead of `[" ", "last"]`. This will impact the probability of predicting the next word if you finish your prompt with a whitespace or not. As evident from the sample output above, you may observe that certain tokens begin with a special character **(Ġ)** representing whitespace, while others lack this feature.

- **Model-specific**: Even though most language models are using BPE method for tokenization, they still train a new tokenizer for their own models. GPT-4, LLaMA, OpenAssistant, and similar models all develop their separate tokenizers.